In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("./datathon_train.csv")

def get_time(string):
  return int(string[:2])
dataset['DEP_TIME'] = dataset['DEP_TIME_BLK'].apply(get_time)

ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
dataset_clean = dataset.drop(['Id', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'DEP_TIME_BLK','DEST'], axis = 1)
dataset_clean.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Assuming 'X_df' is your dataset and 'Departing Airport Name' is a categorical column
categorical_columns = ['PREVIOUS_AIRPORT', 'CARRIER_NAME', 'DEPARTING_AIRPORT']

# Apply one-hot encoding to categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(dataset_clean[categorical_columns])

# Drop the original categorical columns from the original DataFrame
dataset_clean = dataset_clean.drop(categorical_columns, axis=1)

# Convert the one-hot encoded NumPy array back to a DataFrame with feature names
encoded_feature_names = []
for category, column in zip(encoder.categories_, categorical_columns):
    encoded_feature_names.extend([f"{column}_{cat}" for cat in category])
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_feature_names)

# Concatenate the one-hot encoded columns with the original dataset
X_df = pd.concat([dataset_clean, X_encoded_df], axis=1)


In [ ]:
y = X_df['IS_DELAYED'].to_numpy()
New_X_df = X_df.drop(['DEP_DELAY_NEW', 'IS_DELAYED'], axis = 1)
X = New_X_df.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into 60% training, 20% validation, and 20% testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Split the remaining data into 50% validation and 50% testing
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to your training data and transform it
X_train_normalized = scaler.fit_transform(X_train)

# Transform the validation and test data using the same scaler
X_val_normalized = scaler.transform(X_val)
X_test_normalized = scaler.transform(X_test)




In [ ]:
#shuffle the data
from sklearn.utils import shuffle
# Shuffle the training data
X_train_normalized, y_train = shuffle(X_train_normalized, y_train, random_state=42)

# Shuffle the validation data (if needed)
X_val_normalized, y_eval = shuffle(X_val_normalized, y_val, random_state=42)

# Shuffle the development data
X_dev_normalized, y_dev = shuffle(X_test_normalized, y_test, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
model_1 = Sequential()

model_1.add(Dense(50, activation='relu', input_shape=(63,)))

model_1.add(Dense(30, activation='relu'))

model_1.add(Dense(10, activation='relu'))

model_1.add(Dense(1, activation='sigmoid'))

model_1.summary()

model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results1 = model_1.fit(X_train, y_train, epochs=10, batch_size=32, vvalidation_data=(X_val, y_val))
y_pred_m1 = model_1.predict(X_test)
y_pred_m1 =(y_pred_m1 > 0.5)